In [30]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

## **层和块**
* 块包含层，往往块是由多个层进行组合。一般来说，我们用一个顺序块去包住层

In [20]:
net = nn.Sequential( nn.Linear(20 , 64) , nn.ReLU() , nn.Linear( 64 ,10 ))#首先输入的特征往往对应的就是第一个层的第一个参数,输出的就是
                                                                          #神经元的个数，在回归中就是输出点，在卷积中就叫特征图。
                                                                          #总之，第一个参数是输入量，往往对应特征数量；第二个参数对应输
                                                                          #出量，往往对应的是神经元的个数。
x = torch.randn( 5,20) #5个有20个特征的样本
net( x )

tensor([[ 0.1361, -0.0779, -0.1813, -0.4100, -0.4418,  0.0078,  0.3008,  0.0646,
          0.0243, -0.2589],
        [ 0.2090, -0.0438, -0.0329,  0.0540, -0.0679, -0.1013, -0.2931, -0.0102,
          0.0307, -0.0271],
        [ 0.2174,  0.0159, -0.0890, -0.5111, -0.2277,  0.0355,  0.3200,  0.0702,
         -0.0799,  0.2355],
        [-0.1442, -0.2264, -0.1001, -0.0560, -0.2702, -0.4244, -0.2735,  0.2239,
         -0.2212, -0.0164],
        [-0.0072,  0.1311, -0.1069, -0.0607, -0.2996,  0.0043,  0.1878,  0.2174,
         -0.2093,  0.2599]], grad_fn=<AddmmBackward>)

## **自定义块**
* 手动实现一下sequential类

In [4]:
class Mysequential( nn.Module ):
    def __init__( self  , *args ):#传入可变参数进行初始化
        #需要继承一下Module类的初始化方法和属性
        super().__init__()#通过阅读源码知道在Module的初始化中，实现了self._modules = OrderedDict()，这个_modules是一个有序字典
        for layer in args:
            self._modules[layer] = layer#累加层到一个顺序字典里，实现顺序块的关键步骤
    def forward( self , X ):
        for layer in self._modules:
            X = layer( X )
        return X

In [14]:
net = Mysequential( nn.Linear(20 , 64) , nn.ReLU() , nn.Linear( 64 ,10 ))
x = torch.rand( 5,20) #5个有20个特征的样本
net( x )

tensor([[ 0.0139, -0.0791, -0.1985,  0.3493,  0.0963, -0.1272, -0.5149,  0.2642,
          0.3270,  0.2131],
        [ 0.0139, -0.0791, -0.1985,  0.3493,  0.0963, -0.1272, -0.5149,  0.2642,
          0.3270,  0.2131],
        [ 0.0139, -0.0791, -0.1985,  0.3493,  0.0963, -0.1272, -0.5149,  0.2642,
          0.3270,  0.2131],
        [ 0.0139, -0.0791, -0.1985,  0.3493,  0.0963, -0.1272, -0.5149,  0.2642,
          0.3270,  0.2131],
        [ 0.0139, -0.0791, -0.1985,  0.3493,  0.0963, -0.1272, -0.5149,  0.2642,
          0.3270,  0.2131]], grad_fn=<AddmmBackward>)

## 自定义参数

In [33]:
class FixedHiddenMLP( nn.Module ):
    def __init__( self ):
        super().__init__()
        self.contant_weight = torch.rand( 20, 10 ,requires_grad= False )#requires_grad= False,在后向传播当中这个权值参数就是常量，
                                                                        #不会被更新
        self.layer = nn.Linear( 10 , 20 )
    def forward(self, X):
        X = self.layer( X )
        X = torch.mm( X , self.contant_weight ) + 1 
        X = F.relu( X ) #F是在定义当中的functional类，主要是对一些层的函数实现，这样就可以在前向传播函数当中直接对输入量进行操作，
                        #就不用在初始化中再加入Relu层，这样的话就可以不对初始化模型进行修改
        X = self.layer( X )#这里复用layer层，那么这个层的参数与上面那个层的参数是一致的。

        return X

In [34]:
X = torch.rand( 5 , 10 )
net = FixedHiddenMLP()
net( X )

tensor([[ 0.0546, -0.7777,  0.5692, -0.6908,  0.2741,  0.1010, -0.4682,  0.1102,
         -0.2919,  0.3848, -0.9035,  0.4315,  0.0947,  0.7482,  0.6218, -0.1352,
          0.0287, -0.4739,  0.2516, -0.2151],
        [ 0.0907, -0.7555,  0.5060, -0.5949,  0.3715,  0.0277, -0.3956,  0.3200,
         -0.2076,  0.4329, -0.9733,  0.2723,  0.2725,  0.6662,  0.5838, -0.0892,
         -0.0227, -0.3374,  0.1374, -0.3169],
        [ 0.1827, -0.9509,  0.9284, -0.9309, -0.0571, -0.0245, -0.2456,  0.3904,
         -0.3647,  0.5307, -1.1749,  0.4812,  0.3225,  0.9593,  1.0047, -0.1434,
          0.2081, -0.6573,  0.3745, -0.2868],
        [ 0.0790, -0.7212,  0.5641, -0.6688, -0.0278, -0.0078, -0.1620,  0.3352,
         -0.3204,  0.4468, -1.1162,  0.1972,  0.4394,  0.6649,  0.9002,  0.1215,
         -0.1894, -0.3243,  0.1685, -0.1695],
        [ 0.0305, -0.5402,  0.3592, -0.3824,  0.3263,  0.0122, -0.2735,  0.3257,
         -0.1666,  0.3761, -0.7753,  0.0690,  0.4220,  0.4699,  0.4999,  0.0150,
      

## **块与块的嵌套**

In [38]:
class NetWork( nn.Module ):
    def __init__( self ):
        super().__init__()
        self.linear1 = nn.Sequential( nn.Linear( 10 ,30 ), nn.ReLU() , 
                                      nn.Linear( 30 , 10 ), nn.ReLU() )
    def forward( self , X ):
        X  = self.linear1(X )
        return X

In [39]:
X = torch.rand( 4,10)
net  = nn.Sequential( NetWork( ) , FixedHiddenMLP() )#复用块,应该输出的是FixedHiddenMLP()最后的输出量，也就是一个样本10个特征输入，
                                                     #20个输出
net( X )

tensor([[-0.0920, -0.3830, -0.3305,  1.6995,  0.9109, -0.1870,  1.2934,  0.7499,
          1.8255, -0.7216,  0.4341,  0.0480,  1.8688, -0.5703, -1.1804,  0.0307,
         -0.2616,  0.4715, -0.0387,  1.6970],
        [-0.0937, -0.3668, -0.3192,  1.6451,  0.8807, -0.1920,  1.2703,  0.7341,
          1.7716, -0.6946,  0.4249,  0.0519,  1.8218, -0.5557, -1.1441,  0.0362,
         -0.2448,  0.4776, -0.0429,  1.6557],
        [-0.0950, -0.3783, -0.3221,  1.6820,  0.8872, -0.1941,  1.2722,  0.7409,
          1.7958, -0.7080,  0.4246,  0.0470,  1.8338, -0.5660, -1.1647,  0.0255,
         -0.2566,  0.4633, -0.0480,  1.6724],
        [-0.0957, -0.3750, -0.3112,  1.6530,  0.8672, -0.2017,  1.2637,  0.7335,
          1.7694, -0.7054,  0.4145,  0.0539,  1.8125, -0.5589, -1.1369,  0.0331,
         -0.2469,  0.4659, -0.0520,  1.6508]], grad_fn=<AddmmBackward>)